# Imports

In [1]:
import tensorflow as tf
from matplotlib import pyplot as plt
from PIL import Image


import os
import cv2
import random
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from imutils import paths

# Importing and Preprocessing Data

In [15]:
import gc

image_height = 75
image_width = 75
channel = 1 #For RGB images, 1 for greyscale images
input_imageShape = (image_height, image_width, channel) 
input_imageSize = (image_height, image_width)        

def load_data(image_dir, csv_filename):
    image_data = []
    class_labels = []    
    imagePaths = sorted(list(paths.list_images(image_dir)))             #To load images from the respective folder
    df = pd.read_csv(csv_filename, header = None, index_col = False, engine="python")  #To read class labels from .csv file
    random.seed(42)                                                     #Seed value can be modified accordingly
    random.shuffle(imagePaths)
    print("[INFO] loading images and class labels...")
    for imagePath in imagePaths:
        image = cv2.imread(imagePath)
        image = cv2.resize(image, input_imageSize)    #Uncomment if required....
        image = img_to_array(image)
        image_data.append(image)    
        fullname = os.path.basename(imagePath)        #Complete image name with extension e.g. "1.png" etc.
        image_name = os.path.splitext(fullname)[0]    #Removing extension i.e. "1" etc.
        image_name = image_name.split("(")[0]
        temp=df[df[0] == int(image_name)]             #Finding image name in respective .csv file
        label = temp.iloc[0,1]                        #Finding respective class label 
        class_labels.append(label)
    print("[INFO] Creating Numpy Arrays...")
    X = np.array(image_data, dtype="float32")                        #Creating required Numpy array of images
    Y = np.array(class_labels)                      #Creating required Numpy array of class labels
    
    # converting from 3 color channels to 1 channel
    X = X[:,:,:,0]
    X = X.reshape(X.shape+(1,))

    # free memory explicitly
    image_data = None
    class_labels = None
    gc.collect()
    
    return X, Y

In [16]:
#some required values
number_of_classes = 250

In [17]:
# loading training data
image_dir = r"D:\Codes\Github\Projects\competitions\Sketch-Recognizer\Scripts\Data\Train"
csv_filename = r"D:\Codes\Github\Projects\competitions\Sketch-Recognizer\Scripts\Data\Train.csv"
(x_train, y_train) = load_data(image_dir, csv_filename)

# load the validation data
val_image_dir = r"D:\Codes\Github\Projects\competitions\Sketch-Recognizer\Scripts\Data\Validation"
val_csv_filename = r"D:\Codes\Github\Projects\competitions\Sketch-Recognizer\Scripts\Data\Validation.csv"
(x_test, y_test) = load_data(val_image_dir, val_csv_filename)

[INFO] loading images and class labels...
[INFO] Creating Numpy Arrays...
[INFO] loading images and class labels...
[INFO] Creating Numpy Arrays...


In [18]:
# inspecting the shape
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (10004, 75, 75, 1)
10004 train samples
5000 test samples


In [19]:
# Normalise the data and convert to float32
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [20]:
# expand dimenstion to (n, 1) for next step
y_train = np.expand_dims(y_train, 1)
y_test = np.expand_dims(y_test, 1)

In [21]:
# adding 4000 validation set examples to training set
x_train = np.concatenate((x_train, x_test[1000:]))
y_train = np.concatenate((y_train, y_test[1000:]))

x_test = x_test[0:1000]
y_test = y_test[0:1000]
gc.collect()

54

In [22]:
# Convert labels to one-hot vectors
y_train = tf.one_hot(y_train, number_of_classes) # or use tf.one_hot()
y_test = tf.one_hot(y_test, number_of_classes)

In [23]:
# Removing extra dimension from one-hot vectors
output_list = []

for i in range(y_train.shape[0]):
    output_list.append(y_train[i][0])

y_train = tf.stack(output_list)

output_list = []

for i in range(y_test.shape[0]):
    output_list.append(y_test[i][0])

y_test = tf.stack(output_list)

# Beginning  the Game with ResNet V2 

- Model Definition
- Data Augmentation Options
- Model Training 
- Model Evaluation

In [45]:
# Using ResNet V2
from __future__ import print_function
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import numpy as np
import os

# Training parameters
batch_size = 128  # orig paper trained all networks with batch_size=128
epochs = 10
data_augmentation = True
num_classes = number_of_classes

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# w.r.t Cifar10 dataset
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 3

depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dV2' % (depth)

# Input image dimensions.
input_shape = x_train.shape[1:]

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-4
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

def resnet_v2(input_shape, depth, num_classes=250):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

# instantiating model
model = resnet_v2(input_shape=input_shape, depth=depth)

# compiling model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])

model.summary()
print(model_type)

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=50,
        # randomly shift images horizontally
        width_shift_range=0.3,
        # randomly shift images vertically
        height_shift_range=0.3,
        # set range for random shear
        shear_range=0.3,
        # set range for random zoom
        zoom_range=0.2,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

x_train shape: (14004, 75, 75, 1)
14004 train samples
1000 test samples
y_train shape: (14004, 250)
Learning rate:  0.0001
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 75, 75, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_62 (Conv2D)              (None, 75, 75, 16)   160         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_56 (BatchNo (None, 75, 75, 16)   64          conv2d_62[0][0]                  
__________________________________________________________________________________________________
activation_56 (Activation)      (None, 75, 75, 16)   0           bat

conv2d_83 (Conv2D)              (None, 19, 19, 128)  16512       activation_74[0][0]              
__________________________________________________________________________________________________
batch_normalization_75 (BatchNo (None, 19, 19, 128)  512         conv2d_83[0][0]                  
__________________________________________________________________________________________________
activation_75 (Activation)      (None, 19, 19, 128)  0           batch_normalization_75[0][0]     
__________________________________________________________________________________________________
conv2d_84 (Conv2D)              (None, 19, 19, 128)  147584      activation_75[0][0]              
__________________________________________________________________________________________________
batch_normalization_76 (BatchNo (None, 19, 19, 128)  512         conv2d_84[0][0]                  
__________________________________________________________________________________________________
activation

In [49]:
# Fit the model using data augmentation on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train, batch_size=128),
                    validation_data=(x_test, y_test),
                    epochs=20, verbose=1,
                    callbacks=callbacks)

Learning rate:  0.0001
Epoch 1/20
 44/110 [===========>..................] - ETA: 4:11 - loss: 1.8177 - accuracy: 0.6026

KeyboardInterrupt: 

In [ ]:
# Fit the model without data augmentation
model.fit(x_train, y_train, batch_size=128,
                    validation_data=(x_test, y_test),
                    epochs=epochs, verbose=1,
                    callbacks=callbacks).Z

In [50]:
# Evaluate  our trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

1000/1000 [==============================] - 2s 2ms/sample - loss: 2.9230 - accuracy: 0.4770
Test loss: 2.9229703369140627
Test accuracy: 0.477


# Saving Models, Loading Models & Making Predictions

In [44]:
# saving weights
model.save_weights(r"D:\ML\Weights\ResNet V2\temp.h5")

In [ ]:
# saving model
model.save(r"D:\ML\Weights\ResNet V2\drawing250_28x28\drawings250_model_final.h5")

In [ ]:
# loading model
model = tf.keras.models.load_model(r"D:\ML\Weights\ResNet V2\drawings250_model_last.h5")
model.summary()

In [47]:
# loading weights
model.load_weights(r"D:\ML\Weights\ResNet V2\temp.h5")

In [36]:
# making predictions on given indices in test set
start = int(input("Enter starting index: "))
end = int(input("Enter ending index (won't be included): "))

# getting predicted class integers
indices = tf.argmax(input=model.predict(x_test[start:end]),axis=1)
print(indices.numpy())

#printing actual class integers
print("[", end="")
for vector in y_test[start:end]:
    print(np.argmax(vector), end=" ")
print("]", end="")

Enter starting index: 5
Enter ending index (won't be included): 10
[117 145 142 138 184]
[168 145 142 138 157 ]